# Composio - ClickUp Actions Test

In [1]:
# import os
# import yaml
# from langchain.agents import create_tool_calling_agent, AgentExecutor
# from langchain_openai import ChatOpenAI
# from langchain_core.prompts import ChatPromptTemplate
from composio_langchain import ComposioToolSet, App, Action
# from langchain.memory import ConversationBufferMemory
# from langchain.tools import StructuredTool
# from langchain_core.tools import ToolException
from typing import List, Optional, Dict, Any, Tuple
# from pydantic import ValidationError
from pprint import pprint
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
def test_composio_clickup_tools(actions_with_args: List[Dict[str, Any]], workspace_id = 12927880) -> Tuple[List[Dict[str, Any]], List[Dict[str, Any]]]:
    """
    Test Composio ClickUp tools with given actions and arguments.

    Args:
        actions_with_args (List[Dict[str, Any]]): A list of dictionaries containing action names and their arguments.

    Returns:
        Tuple[List[Dict[str, Any]], List[Dict[str, Any]]]: A tuple containing two lists:
            - short_results: A summary of executed actions.
            - full_results: Detailed results of executed actions.
    """
    start_time = datetime.now()
    composio_toolset = ComposioToolSet(workspace_id=workspace_id)
    short_report = []
    long_report = []

    for action in actions_with_args:
        try:
            clickup_action = composio_toolset.get_actions(actions=[getattr(Action, action["name"])])
            result = clickup_action[0].invoke(action["args"])
            short_report.append({"action": action["name"], "executed": result['execution_details']['executed']})
            long_report.append({"action": action["name"], "results": result})
        except Exception as e:
            short_report.append({"action": action["name"], "executed": result['execution_details']['executed']})
            long_report.append({"action": action["name"], "executed": result['execution_details']['executed'], "error": str(e)})
        print(short_report[-1])
    
    failed_count = sum(1 for report in short_report if not report["executed"])
    total_actions = len(actions_with_args)
    print(f"\n{failed_count} out of {total_actions} actions failed")
    
    end_time = datetime.now()
    print(f"\nTest Duration: {end_time - start_time}")
    
    return short_report, long_report

def print_report(report):
    divider = "=" * 100
    pprint(report)
    print("\n", divider, "\n")

def print_failed(long_report):
    failed_found = False
    for report in long_report:
        try:
            if report["results"]["execution_details"]['executed'] == False:
                print_report(report)
                failed_found = True
        except KeyError:
            if "executed" in report and report["executed"] == False:
                print_report(report)
                failed_found = True
    if not failed_found:
        print("All actions executed successfully")

---

## Lists Test

In [3]:
# Arguments values
list_id = 901304862476
folder_id = 54804921
list_name = f"-_-_ NEW LIST CREATED AT {datetime.now()}  _-_-"

# Actions with arguments
list_actions = [
    {
    "name": "CLICKUP_GET_LIST",
    "args": {
        "list_id": list_id
    }
    },
    {
        "name": "CLICKUP_GET_LISTS",
        "args": {
            "folder_id": folder_id
        }
    },
    {
        "name": "CLICKUP_CREATE_LIST",
        "args": {
            "folder_id": folder_id, 
            "name": list_name,
        }
    },
    {
        "name": "CLICKUP_MEMBERS_GET_LIST_USERS",
        "args": {
            "list_id": list_id,
        }
    },
]

In [4]:
short_report_1, long_report_1 = test_composio_clickup_tools(list_actions)

{'action': 'CLICKUP_GET_LIST', 'executed': True}
{'action': 'CLICKUP_GET_LISTS', 'executed': False}
{'action': 'CLICKUP_CREATE_LIST', 'executed': False}
{'action': 'CLICKUP_MEMBERS_GET_LIST_USERS', 'executed': True}

2 out of 4 actions failed

Test Duration: 0:00:14.169768


In [25]:
print_failed(long_report_1)

{'action': 'CLICKUP_GET_LISTS',
 'results': {'execution_details': {'executed': False},
             'response_data': {'error': '{"err":"Team not '
                                        'authorized","ECODE":"OAUTH_027"}',
                               'status_code': 401}}}


{'action': 'CLICKUP_CREATE_LIST',
 'results': {'execution_details': {'executed': False},
             'response_data': {'error': '{"err":"Team not '
                                        'authorized","ECODE":"OAUTH_027"}',
                               'status_code': 401}}}




### Conclusion

It's not clear why we were unable to get lists or create lists. It would be useful to have more detailed error messages

---

<br>

## Spaces / Folders Test

In [6]:
team_id = 12927880
folder_id = "54804921"
space_id = 54804921

spaces_and_folders_actions = [
    {
        "name": "CLICKUP_AUTHORIZATION_VIEW_ACCOUNT_DETAILS",
        "args": {}
    },
    {
        "name": "CLICKUP_AUTHORIZATION_GET_WORK_SPACE_LIST",
        "args": {}
    },
   
    {
        "name": "CLICKUP_SPACES_GET_SPACE_DETAILS",
        "args": {
            "team_id": team_id,
        }
    },
    {
        "name": "CLICKUP_GET_WORKSPACE_SEATS",
        "args": {
            "team_id": team_id,
        }
    },
    {
        "name": "CLICKUP_GET_SPACES",
        "args": {
            "team_id": team_id,
        }
    },
    {
        "name": "CLICKUP_GET_SPACE",
        "args": {
            "space_id": space_id,
        }
    },
    {
        "name": "CLICKUP_GET_FOLDERS",
        "args": {
            "space_id": space_id,
        }
    },
]

In [7]:
short_report_2, long_report_2 = test_composio_clickup_tools(spaces_and_folders_actions)

{'action': 'CLICKUP_AUTHORIZATION_VIEW_ACCOUNT_DETAILS', 'executed': True}
{'action': 'CLICKUP_AUTHORIZATION_GET_WORK_SPACE_LIST', 'executed': True}
{'action': 'CLICKUP_SPACES_GET_SPACE_DETAILS', 'executed': True}
{'action': 'CLICKUP_GET_WORKSPACE_SEATS', 'executed': True}
{'action': 'CLICKUP_GET_SPACES', 'executed': True}
{'action': 'CLICKUP_GET_SPACE', 'executed': True}
{'action': 'CLICKUP_GET_FOLDERS', 'executed': True}

0 out of 7 actions failed

Test Duration: 0:00:24.019839


In [8]:
print_failed(long_report_2)

All actions executed successfully


### Conclusion

All actions related to workspace, spaces and folders that were tested here seem to be working correctly

---

<br>

## Tasks Test

In [20]:
list_id = 901304862476
list_id_remove_task_from_list = 901304890424
task_name = f"-_-_ COMPOSIO CLICKUP CREATE TASK TEST - {datetime.now()} _-_-"
updated_task_name =  f"-_-_ COMPOSIO CLICKUP UPDATE TASK TEST - {datetime.now()} _-_-"
task_id = "86a4d5k4j"
# task_id_delete = ["86a4d5k3y", "86a4d5k4b", "86a4d5k4j"]
task_id_delete = "86a4d5k3c"
task_id_remove_from_list = "86a4d9kxx"
comment_text = "Comentário feito a partir da ação 'CLICKUP_CREATE_TASK_COMMENT'"
assignee = int(82061927)
notify_all = False


tasks_actions = [
    {
        "name": "CLICKUP_CREATE_TASK_ATTACHMENT",
        "args": {
            "task_id": task_id,
            }
    },
    {
        "name": "CLICKUP_CREATE_TASK_COMMENT",
        "args": {
            "task_id": task_id, 
            "comment_text": comment_text, 
            "assignee": assignee, 
            "notify_all": notify_all
        }
    },
    {
        "name": "CLICKUP_GET_TASK_COMMENTS",
        "args": {
            "task_id": task_id,
        }
    },    
    {
        "name": "CLICKUP_ADD_TASK_TO_LIST",
        "args": {
            "list_id": list_id, 
            "task_id": task_id
        }
    },
    {
        "name": "CLICKUP_REMOVE_TASK_FROM_LIST",
        "args": {
            "list_id": list_id_remove_task_from_list, 
            "task_id": task_id_remove_from_list
        }
    },
     {
        "name": "CLICKUP_CREATE_TASK",
        "args": {
            "list_id": list_id,
            "name": task_name,
        }
    },    
    {
        "name": "CLICKUP_GET_TASK",
        "args": {
            "task_id": task_id,
        }
    },
    {
        "name": "CLICKUP_UPDATE_TASK",
        "args": {
            "task_id": task_id,
            "name": updated_task_name,
            "assignees__add": [],
            "assignees__rem": [],
        }
    },
    {
        "name": "CLICKUP_DELETE_TASK",
        "args": {
            "task_id": task_id_delete,
        }
    },
    {
        "name": "CLICKUP_GET_TASKS",
        "args": {
            "list_id": list_id,
        }
    },
    {
        "name": "CLICKUP_GET_FILTERED_TEAM_TASKS",
        "args": {
            "team_id": team_id,

        }
    },
    {
        "name": "CLICKUP_GET_TASK_MEMBERS",
        "args": {
            "task_id": task_id,
        }
    },
]

In [21]:
short_report_3, long_report_3 = test_composio_clickup_tools(tasks_actions)

{'action': 'CLICKUP_CREATE_TASK_ATTACHMENT', 'executed': False}
{'action': 'CLICKUP_CREATE_TASK_COMMENT', 'executed': True}
{'action': 'CLICKUP_GET_TASK_COMMENTS', 'executed': True}
{'action': 'CLICKUP_ADD_TASK_TO_LIST', 'executed': True}
{'action': 'CLICKUP_REMOVE_TASK_FROM_LIST', 'executed': True}
{'action': 'CLICKUP_CREATE_TASK', 'executed': True}
{'action': 'CLICKUP_GET_TASK', 'executed': True}
{'action': 'CLICKUP_UPDATE_TASK', 'executed': True}
{'action': 'CLICKUP_DELETE_TASK', 'executed': True}
{'action': 'CLICKUP_GET_TASKS', 'executed': True}
{'action': 'CLICKUP_GET_FILTERED_TEAM_TASKS', 'executed': True}
{'action': 'CLICKUP_GET_TASK_MEMBERS', 'executed': True}

1 out of 12 actions failed

Test Duration: 0:00:35.891557


In [22]:
print_failed(long_report_3)

{'action': 'CLICKUP_CREATE_TASK_ATTACHMENT',
 'results': {'execution_details': {'executed': False},
             'response_data': {'error': '{"err":"Request is not '
                                        '\'multipart/form-data\'","ECODE":"ATTCH_045"}',
                               'status_code': 400}}}




### Conclusion

This action doesn't provide a way to actually attach something to a task. We suspect that the tool needs to accept another argument where the user can somehow select the file he or she intends to attach

---

<br>

## Guest / User Test

In [15]:
invite_guest_action = [
    {
        "name": "CLICKUP_INVITE_GUEST_TO_WORKSPACE",
        "args": {
            "team_id": team_id,
            "email": "joaomorossini@duck.com", 
            "can_edit": False, 
            "can_see_time_spent": False,
            "can_edit_tags": False,
            "can_see_time_estimated": False,
            "can_create_views": False,
            "custom_role_id": 0,
        }
    }
]

In [12]:
list_id = 901103926955
email = "joaomorossini@duck.com"
guest_id = 
user_id_delete = 
task_id = ""

guest_and_user_actions = [
    {
        "name": "CLICKUP_INVITE_GUEST_TO_WORKSPACE",
        "args": {
            "team_id": team_id, 
            "email": email, 
            "can_edit": False, 
            "can_see_time_spent": False,
            "can_edit_tags": False,
            "can_see_time_estimated": False,
            "can_create_views": False,
            "custom_role_id": 0,
        }
    },
    {
        "name": "CLICKUP_REMOVE_GUEST_FROM_WORKSPACE",
        "args": {
            "team_id": team_id, 
            "guest_id": guest_id
        }
    },
    {
        "name": "CLICKUP_ADD_GUEST_TO_TASK",
        "args": {
            "task_id": task_id, 
            "guest_id": guest_id, 
            "permission_level": 'read'
        }
    },
    {
        "name": "CLICKUP_REMOVE_GUEST_FROM_TASK",
        "args": {
            "task_id": task_id,
            "guest_id": guest_id
        }
    },
    {
        "name": "CLICKUP_ADD_GUEST_TO_LIST",
        "args": {
            "list_id": list_id, 
            "guest_id": guest_id, 
            "permission_level": 'read'
        }
    },
    {
        "name": "CLICKUP_REMOVE_GUEST_FROM_LIST",
        "args": {
            "list_id": list_id, 
            "guest_id": guest_id
        }
    },
    {
        "name": "CLICKUP_INVITE_USER_TO_WORKSPACE",
        "args": {
            "team_id": team_id,
            "email": email,
            "admin": False,
        }
    },
    {
        "name": "CLICKUP_REMOVE_USER_FROM_WORKSPACE",
        "args": {
            "team_id": team_id,
            "user_id": user_id_delete,
        }
    },
]

In [18]:
short_report_4, long_report_4 = test_composio_clickup_tools(guest_and_user_actions, workspace_id=team_id)

{'action': 'CLICKUP_INVITE_GUEST_TO_WORKSPACE', 'executed': False}
{'action': 'CLICKUP_REMOVE_GUEST_FROM_WORKSPACE', 'executed': False}
{'action': 'CLICKUP_ADD_GUEST_TO_TASK', 'executed': False}
{'action': 'CLICKUP_REMOVE_GUEST_FROM_TASK', 'executed': False}
{'action': 'CLICKUP_ADD_GUEST_TO_LIST', 'executed': False}
{'action': 'CLICKUP_REMOVE_GUEST_FROM_LIST', 'executed': False}
{'action': 'CLICKUP_INVITE_USER_TO_WORKSPACE', 'executed': False}
{'action': 'CLICKUP_REMOVE_USER_FROM_WORKSPACE', 'executed': False}

8 out of 8 actions failed

Test Duration: 0:00:25.903528


In [19]:
print_failed(long_report_4)

{'action': 'CLICKUP_INVITE_GUEST_TO_WORKSPACE',
 'results': {'execution_details': {'executed': False},
             'response_data': {'error': '{"err":"Team not '
                                        'authorized","ECODE":"OAUTH_057"}',
                               'status_code': 401}}}


{'action': 'CLICKUP_REMOVE_GUEST_FROM_WORKSPACE',
 'results': {'execution_details': {'executed': False},
             'response_data': {'error': '{"err":"Team not '
                                        'authorized","ECODE":"OAUTH_057"}',
                               'status_code': 401}}}


{'action': 'CLICKUP_ADD_GUEST_TO_TASK',
 'results': {'execution_details': {'executed': False},
             'response_data': {'error': '{"err":"Team(s) not '
                                        'authorized","ECODE":"OAUTH_023"}',
                               'status_code': 401}}}


{'action': 'CLICKUP_REMOVE_GUEST_FROM_TASK',
 'results': {'execution_details': {'executed': False},
             'respons

### Conclusion

It's not clear why we were unable to use actions that need guest ids and user ids to work. It would be useful to have more detailed error messages.

CLICKUP_INVITE_USER_TO_WORKSPACE: The error message states that a required field is missing, but the official docs do not show ```list_id``` as an argument for this action.

CLICKUP_REMOVE_USER_FROM_WORKSPACE: We're on the free plan, so that seems to be the reason why we couldn't execute the action

---

<br>

In [ ]:
successful_actions = []

for report in [long_report_1, long_report_2, long_report_3, long_report_4, long_report_5]:
    for action in report:
        if action['status'] == 'success':
            successful_actions.append(action['name'])

print(successful_actions)
